## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [29]:
import numpy as np
import pandas as pd
import random
import string

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [30]:
import random

N = 1000000
A = [random.randint(0, 1000) for i in range(N)]
B = [a + 100 for a in A]

average_B = sum(B) / len(B)
print("Среднее значение массива B: ", average_B)


Среднее значение массива B:  600.299318


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [31]:
key=[''.join(random.choices(string.ascii_letters, k=5)) for _ in range(2000000)]
table=pd.DataFrame(np.random.randint(0, 1001, size=(2000000,4)), index=key)
table

,0,1,2,3
UGjhR,437,270,647,870
KEbwV,810,997,477,91
VpcXR,904,621,683,258
fwCKs,859,598,593,578
wbuOM,512,677,197,505
...,...,...,...,...
WsQSi,417,694,376,865
FnHBo,524,174,967,450
IXqpb,444,285,942,722
jcEDL,540,162,118,715


In [57]:
table.loc['FnHBo']

0    524
1    174
2    967
3    450
Name: FnHBo, dtype: int32

In [58]:
try:
    table.loc['abcde']
except KeyError:
    print('нет key, где указаны первые 5 английских букв')

нет key, где указаны первые 5 английских букв


## Лабораторная работа 3

In [37]:
!pip install line_profiler

Defaulting to user installation because normal site-packages is not writeable


In [38]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [39]:
import timeit

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [40]:
import pandas as pd

In [41]:
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
reviews = pd.read_csv('reviews_sample.csv', delimiter=',')
reviews.rename(columns = {'Unnamed: 0':'id'}, inplace = True )
df=pd.merge(recipes, reviews)
df['submitted'] = pd.to_datetime(df['submitted'])

In [42]:
df


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,user_id,recipe_id,date,rating,review
0,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,2000576480,445211,2015-10-18,1,Terrible - makes a very runny batter like for ...
1,say what banana sandwich,95926,5,118163,2004-07-20,4.0,you just have to try it to believe it.,NaN,221694,126623,2009-02-01,4,This was yummy. I cut the entire sauce recipe ...
2,add in anything muffins,149593,15,89831,2005-12-28,NaN,"this is a never-fail muffin recipe, it's a bla...",9.0,52282,66815,2008-07-12,5,"my family all enjoyed this dish, and yes it do..."
3,burek or feta cheese phyllo pie,310570,65,676820,2008-06-24,38.0,"ok, there are different version of burek (some...",6.0,527607,260529,2007-10-23,5,These were fantastic. I lightened it even fur...
4,skordy new potatoes w rosemary lemon olive oi,296983,35,718054,2008-04-08,NaN,i took this recipe from a vegan tastes of gree...,6.0,795795,50385,2008-03-20,5,This recipe was WONDERFULLY DELICIOUS! The rev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446,zucchini meat sauce with pasta,253264,30,275742,2007-09-17,NaN,havn't tried this recipe yet but hopefully wil...,6.0,142849,102617,2007-08-01,5,This is fabulous! I do cheat however and use t...
3447,zucchini stuffed with feta and basil,72857,30,95743,2003-10-08,NaN,"easy to make, savory and a nice presentation. ...",5.0,49937,54269,2003-04-20,5,This cake mix extender is great! I used it wit...
3448,zucchini tomato salsa,321224,20,464080,2008-08-25,4.0,a contest winner from taste of home. it state...,13.0,171790,394467,2010-01-11,5,I loved this soup! The only changes I made wer...
3449,zucchini potato and herb fritters,344542,30,197023,2008-12-21,8.0,adapted from a recipe in 'the australian women...,12.0,659599,222188,2013-10-15,5,So simple but yet so delicious. I only made h...


In [43]:

#A
def average():
    total_rating = 0
    count = 0
    for index, row in df.iterrows():
        if row['submitted'].year >= 2010:
            total_rating += row['rating']
            count += 1
            average_rating = total_rating / count
    return average_rating
%prun av=average() 
print(av)
execution_time=timeit.timeit(average, number=1)
print(f'Время исполнения - {execution_time}')

 4.371495327102804
Время исполнения - 0.5427889999996296


In [44]:
af2010 = df.loc[df['submitted'].dt.year >= 2010]

In [45]:
#Б

def average1():
    total_rating = 0
    count = 0
    for index, row in af2010.iterrows():
        total_rating += row['rating']
        count += 1
        average_rating = total_rating / count
    return average_rating

%prun av1=average1()
print(av1)
execution_time=timeit.timeit(average1, number=1)
print(f'Время исполнения - {execution_time}')

 4.371495327102804
Время исполнения - 0.056186000000252534


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [46]:
#В
def average2():
    return af2010['rating'].mean()

%prun av2=average2()
print(av2)
execution_time=timeit.timeit(average2, number=1)
print(f'Время исполнения - {execution_time}')

 4.371495327102804
Время исполнения - 0.00023700000019744039


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [47]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

%prun get_word_reviews_count(af2010)

In [48]:
from collections import defaultdict

def get_word_reviews_count(df):
    word_reviews_count = defaultdict(list)
    
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        unique_words = set(words)  # Используем множество для поиска уникальных слов
        
        for word in unique_words:
            word_reviews_count[word].append(row['recipe_id'])
    
    return word_reviews_count

%prun get_word_reviews_count(af2010)

In [10]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [49]:
import numba
import numpy as np

In [50]:
import time

In [51]:
df=df.loc[df['rating'] > 0]

In [52]:
#A
def mape_basic(df):
    ratings = df['rating'].values
    mean_rating = sum(ratings) / len(ratings)
    mape_sum = 0
    
    for rating in ratings:
        mape_sum += abs((rating - mean_rating) / mean_rating)
    
    mape = mape_sum / len(ratings)
    return mape

%time mape_basic(df)

CPU times: total: 31.2 ms
Wall time: 23.9 ms


0.11138533165746958

In [53]:
#B
@numba.njit
def mape_numba(ratings):
    n = len(ratings)
    mean_rating = 0.0
    mape_sum = 0.0
    
    for i in range(n):
        rating = ratings[i]
        mean_rating += rating
    
    mean_rating /= n
    
    for i in range(n):
        rating = ratings[i]
        mape_sum += abs((rating - mean_rating) / mean_rating)
    
    mape = mape_sum / n
    return mape

%time mape_numba(df['rating'].values)

CPU times: total: 516 ms
Wall time: 987 ms


0.11138533165746958

In [54]:
#C
def mape_numpy(df):
    ratings = df['rating'].values
    mean_rating = np.mean(ratings)
    mape_sum = np.sum(np.abs((ratings - mean_rating) / mean_rating))
    mape = mape_sum / len(ratings)
    return mape

%time mape_numpy(df)

CPU times: total: 0 ns
Wall time: 0 ns


0.11138533165746735

In [55]:
#D
@numba.njit
def mape_np_numba(ratings):
    mean_rating = np.mean(ratings)
    mape_sum = 0
    
    for rating in ratings:
        mape_sum += abs((rating - mean_rating) / mean_rating)
    
    mape = mape_sum / len(ratings)
    return mape
%time mape_np_numba(df['rating'].values)

CPU times: total: 422 ms
Wall time: 490 ms


0.11138533165746958

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4